# Have several agents collaborate in a multi-agent hierarchy 🤖🤝🤖
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

> This tutorial is advanced. You should have notions from [this other cookbook](agents) first!

In this notebook we will make a **multi-agent web browser: an agentic system with several agents collaborating to solve problems using the web!**

It will be a simple hierarchy, using a `ManagedAgent` object to wrap the managed web search agent:

```
       +----------------+
       | Manager agent  |
       +----------------+
               |
       ________|_________________
       |                        |
  Code interpreter   +----------------------+
       tool          |    Managed agent     |
                     | +------------------+ |
                     | | Web Search agent | |
                     | +------------------+ |
                     |          |           |
                     |   Web Search tool    |
                     +----------------------+
```
Let's set up this system. 

⚡️ Our agent will be powered by [meta-llama/Meta-Llama-3.1-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct) using `HfApiEngine` class that uses HF's Inference API: the Inference API allows to quickly and easily run any OS model.

Run the line below to install required dependancies:

In [ ]:
!pip install markdownify duckduckgo-search "git+https://github.com/huggingface/transformers.git#egg=transformers[agents]"

In [1]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

model = "meta-llama/Meta-Llama-3.1-70B-Instruct"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


### 🔍 Create a web search tool

For web browsing, we can already use our pre-existing [`DuckDuckGoSearchTool`](https://github.com/huggingface/transformers/blob/main/src/transformers/agents/search.py) tool to provide a Google search equivalent.

But then we will also need to be able to peak into page found by the `DuckDuckGoSearchTool`.

So for this, let's create a new tool using `markdownify`.

In [2]:
from transformers import Tool
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
import re


class VisitPageTool(Tool):
    name = "visit_webpage"
    description = "Visits a wbepage at the given url and returns its content as a markdown string."
    inputs = {
        "url": {
            "type": "text",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "text"

    def forward(self, url: str) -> str:
        try:
            # Send a GET request to the URL
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes

            # Convert the HTML content to Markdown
            markdown_content = md(response.text).strip()

            # Remove multiple line breaks
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

            return markdown_content

        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Ok, now let's initialize and test our tool!

In [3]:
visit_page_tool = VisitPageTool()

print(visit_page_tool("https://en.wikipedia.org/wiki/Hugging_Face"))

Hugging Face \- Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu
move to sidebar
hide

 Navigation
 

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Contact us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")
* [Donate](https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en "Support us by donating to the Wikimedia Foundation")

 Contribute
 

* [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")
* [Learn to edit](/wiki/Help:Introduction "Learn how to edit Wikipedia")
* [Community portal](/wiki/Wikipedia:Communi

## Build our multi-agent system 🤖🤝🤖

First, we create the web agent, with our two web browsing tools : `search` and `visit_page`.

Which configuration to choose for this one?
- We make it a `ReactJsonAgent`, since web browsing is a single-timeline task that does not require parallel tool calls, so JSON tool calling works well for that.
- Also, since sometimes web search requires exploring many pages before finding the correct answer, we prefer to increase the number of `max_iterations`

In [4]:
from transformers.agents import (
    ReactCodeAgent,
    ReactJsonAgent,
    HfApiEngine,
    ManagedAgent,
)
from transformers.agents.search import DuckDuckGoSearchTool

llm_engine = HfApiEngine(model)

web_agent = ReactJsonAgent(
    tools=[DuckDuckGoSearchTool(), VisitPageTool()],
    llm_engine=llm_engine,
    max_iterations=10,
)

In [5]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search_agent",
    description="Runs web searches for you. Give it your query as an argument.",
)

Finally we create a manager agent, and upon initialization we pass our managed agent to it in its `managed_agents` argument.

Since this agent is the one tasked with the planning and thinking, advanced reasoning will be beneficial : so a `ReactCodeAgent` will be the best choice.

In [6]:
manager_agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    managed_agents=[managed_web_agent],
)

That's all! Now let's run our system! We select a question that requires some calculation and 

In [8]:
manager_agent.run("How much money in total did start-up Stripe raise?")

======== New task ========
How much money in total did start-up Stripe raise?
=== Agent thoughts:
Thought: To find the total amount of money raised by Stripe, I will use the search_agent to search for the total funding raised by Stripe.
>>> Agent is executing the code below:
total_funding = search_agent(request="How much money in total did start-up Stripe raise?")
print("Total funding:", total_funding)
====
======== New task ========
You're a helpful agent named 'search_agent'.
You have been submitted this task by your manager.
---
Task:
How much money in total did start-up Stripe raise?
---
You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much information as possible so that they have a clear understanding of the answer.

Your final_answer WILL HAVE to contain these parts:
### 1. Task outcome (short version):
### 2. Task outcome (extremely detailed version):
### 3. Additional context (if relevant):

Put all these in your final_

'According to Crunchbase, Stripe has raised a total of $2.2B in funding over 19 rounds.'

Our agents managed to efficiently collaborate towards solving the task! ✅

💡 You can easily extend this to more agents : one does the code execution, one the web search, one handles file loadings...

🤔💭 One could even think of doing more complex, tree-like hierarchies, with one CEO agent handling multiple middle managers, each with several reports.

We could even add more intermediate layers, and each one adds a bit more friction to ensure the tasks never get done... Ehm wait, no, let's stick with our simple structure.